<a href="https://colab.research.google.com/github/HananeNourMoussa/arabic-ner/blob/master/jsonl_to_conll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install camel-tools --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import json
from camel_tools.tokenizers.word import simple_word_tokenize
import pandas as pd

In [3]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [4]:
with open ('/content/gdrive/MyDrive/annotated_corpus/batch4.jsonl', 'r') as f: 
  lines = f.readlines()

In [7]:
#converting annotated text from doccano in JSONL format to CoNLL

#create dataframe
conll = pd.DataFrame()
conll['token'] = []
conll['tag'] = []

#for each text in JSONL
for line in lines: 
  data = json.loads(line)  
  text = data['text']
  labels = data['label']
  #annotate tokens before first NE with 'O'
  if labels[0][0] != 0: 
    tokens = simple_word_tokenize(text[0:labels[0][0]])
    df = pd.DataFrame({'token': tokens, 'tag': ['O']*len(tokens)})
    conll = conll.append(df, ignore_index = True)
  #Annotate each entity and the non-entities after it
  for i in range (1, len(labels)): 
    #delimite entity
    ent_beg = labels[i][0]
    ent_end = labels[i][1]
    ent = text[ent_beg:ent_end]
    tag = labels[i][2]
    ent_tokens = simple_word_tokenize(ent)
    print(ent)
    #annotate first token in entity
    try: 
      conll = conll.append({'token': ent_tokens[0], 'tag': 'B-'+tag}, ignore_index = True)
    except: 
      pass
    #entity remaining tokens in entity 
    df = pd.DataFrame({'token': ent_tokens[1:len(ent_tokens)], 'tag': ['I-' + tag]*(len(ent_tokens)-1)})
    conll = conll.append(df, ignore_index = True)
    #check if this is the last entity
    if i != len(labels) - 1: 
      tokens = simple_word_tokenize(text[ent_end:labels[i+1][0]])
    else: 
      tokens = simple_word_tokenize(text[ent_end: len(text)])
    #annotate non-entity with 'O'
    df = pd.DataFrame({'token': tokens, 'tag': ['O']*len(tokens)})
    conll = conll.append(df, ignore_index = True)



Uppsala
سّويد
سطوكهولم
ݣوتنبورݣ
مالمو
2018
172،402
أوپيك
لينݣليزية
OPEC
لمنضمة د لبلدان اللي كاتصدّر لپطرول
14
شتنبر
1960
بغداد
لعراق
إيران
لكويت
السعودية
ڤينيزويلا
1965
ڤيينا
لوطريش
13
44
لأوپيك
لأوپيك
أوپيك
13
الجزائر
أنݣولا
غينيا ستيوائية
لݣابون
إيران
لعراق
لكويت
ليبيا
نيجيريا
جمهورية لكونݣو
السعودية
لإمارات
ڤينيزويلا
لأمم لمتحدة
لأوپيك
أوپولو
الصاموية
Upolu
دصاموا
75
ل1125
كيلوميطر
صاموا
صاڤاي
145 ألف
أوپولو
صاڤاي
أپيا
دصاموا
أوپولو
29
شتنبر
2009
20
ليپا
أوب
أوبي
الروسية
Обь
سيبيريا
5410
روسيا
مونݣوليا
كازاخستان
أوب
ألطاي
سيبيريا
كارا
لمحيط لمجمد الشمالي
نوڤوسيبيريسك
روسيا
سيبيريا
غشت
فبراير
تاونات
فاس
مكناس
9856
2014
58
3,1
47
2,1
2
18
وطاݣو
النݣليزية
Otago
لماوري
Ōtākou
نيوزيلاندا
 لݣزيرة دلجانوب
دونيدين
31,241
229,200
2018
كانطربوري
ساوتلاند
أوطاݣو
ؤطاكو
جاپونية
لمغريب
لأنيمي
لمانݣا
تيليجو
Geek
ؤطاكو
أكيو ناكاموري
1983
لؤطاكو
جّاپون
ؤطاكو
لؤطاكو
جّاپون
لأنيميات
لمانݣات
لؤطاكو
2
ين
2003
18
دولار
أوتربات
ميدلت
درعا
تافيلالت
6819
2014
56
4,7
66,7
2,1
2,8
21,2
ؤطو فون بيسمارك
لألمان

In [8]:
conll

,token,tag
0,Uppsala,B-LOC
1,),O
2,هيّا,O
3,رابع,O
4,أكبر,O
...,...,...
17699,:,O
17700,37,B-MISC
17701,",",I-MISC
17702,5,I-MISC


In [9]:
conll.to_csv('/content/gdrive/MyDrive/annotated_corpus/ner-corpus-darija/CoNLL_2.csv')